# Community Peer Explorer Demo

This notebook is an interactive demo of the Locus Peer Explorer package. 
It allows you to find and explore the peers of any MSA in 2015 or 2016. 

The different types of peers that you can explore are :
* **Geographic peers**: finds the n closest MSAs. This is usually what is done by our potential users as they look for their neighboors. 
* **Input peers**: this features allows you to find peers based on the concentration of any FMs. This will return the n MSAs that have the most similar distribution of those FMs compared to your MSA. You can also choose to identify peers based on outcomes.
* **Peers based on FMs or outcomes automatically selected**: for all these appoaches, we automatically select a subset of FMs or Outcomes and then find the MSAs that are the most similar to you on this subset of features using a distance metric. 
    * *Top FMs*: this feature will return the peers based on the first top n FMs that are the most prevalent in your MSA. You will be able to specify n, the number of FMs you want to base the peer identification on.
    * *Distinguishing FMs*: the peers returned here are based on the FMs that are distinguishing in your MSA. It will include the most present FMs but also the least present. FMs are ranked based on how much the concentration in your MSA differs from other MSAs. You then choose how many of those distinguishing features you want to base your analysis on.
    * *Distinguishing outcomes*: as before, the peers returned here are based on the outcomes that are distinguishing for that MSA.
    * *Minimum coverage peers*: this features selects enough FMs so that a given percentage of the total employment in your MSA is covered. As before, the FMs are selected from the most present to the least present. 
    * *Local/Non-local*: this selects only the local FMs or non-local FMs (waiting for that definition).

In [7]:
%matplotlib inline
from ipywidgets import interactive
from ipywidgets import interact,fixed, interact_manual
import ipywidgets as widgets
%cd '/Users/efloch/Locus/peer-explorer/'
import sys
import locuspeerexplorer.peer_finder as find
import locuspeerexplorer.peer_explorer as exp
import locuspeerexplorer.peer_visualizer as vis
import pandas as pd 
import os 
import geoviz.choropleth as choro

df_data = pd.read_csv('/Users/efloch/Locus/peer-explorer/data/processed/metrics_outcomes.csv')
df_msa_def = pd.read_csv("/Users/efloch/Locus/peer-explorer/data/external/omb_msa_1990_2018.csv")
df_county_dist = pd.read_csv("/Users/efloch/Locus/peer-explorer/data/processed/sf12010countydistance500miles.csv")

def code2name(code):
    return list(df_msa_def[df_msa_def['CBSA_CODE']==code].head(1).CBSA_TITLE)[0]

def name2code(name):
    df_name = df_msa_def[df_msa_def['CBSA_TITLE'].str.contains(name)]
    return list(df_name.head(1).CBSA_CODE)[0]

all_msas = df_msa_def.set_index('CBSA_TITLE').to_dict()['CBSA_CODE']
all_fms = {' '.join((c.split('-')[0]).split('_')):c.split('-')[0] for c in df_data.columns if 'PC_EMPL' in c}
all_fms['None'] = None
all_outcomes = {c:c for c in list(df_data.columns)[3:] if ('-' not in c)}
all_outcomes['None'] = None

MSA='New York'
input_n_peers = widgets.IntSlider(value=5,min=0,max=20,step=1,description='# of Peers',orientation='horizontal')
input_n_features = widgets.IntSlider(value=5,min=0,max=30,step=1,description='# of Features',orientation='horizontal')
input_year = widgets.RadioButtons(options=[2015, 2016],description='Year', disabled=False)
input_msa = widgets.Dropdown(options=all_msas,value=name2code(MSA),description='MSA',)
input_fms = widgets.Dropdown(options=all_fms,description='FM')
input_outcomes = widgets.Dropdown(options=all_outcomes,description='Outcome')


/Users/efloch/Locus/peer-explorer


# Geographic Peers

In [8]:
def show_peers(df_data, df_county_dist, df_msa_def,
                         msa, n_peers, year):
    peers, fms = find.get_geographic_peers(df_data, df_county_dist, df_msa_def,
                         msa, n_peers, year)
    peers.append(msa)
    df_peers = pd.DataFrame({'MSA':peers})
    df_peers['Name'] = df_peers['MSA'].apply(code2name)
#     df_peers['Is peer'] = 1
#     choro.plot(df_peers, 'MSA', 'cbsa code', 'Is peer', 'sequential')

    return df_peers

interact_manual(show_peers, df_data=fixed(df_data), 
                     df_county_dist=fixed(df_county_dist), 
                     df_msa_def= fixed(df_msa_def),
                     msa=input_msa, 
                     n_peers=input_n_peers, year=input_year)

interactive(children=(Dropdown(description='MSA', index=237, options={'Abilene, TX ': 10180, 'Aguadilla-Isabel…

<function __main__.show_peers(df_data, df_county_dist, df_msa_def, msa, n_peers, year)>

# Input FMs peers

In [9]:
def show_fms_peers(df_data, msa, year, n_peers, fms, outcomes):
    if fms is None: fms=[]
    if outcomes is None : ouctomes = []
    peers, fms = find.get_peers_from_input(df_data, msa, year, n_peers, [fms], [outcomes])
    vis.quadrant_viz(df_data, msa, [msa]+peers, fms[0], save_fig=False, show=True)

interact_manual(show_fms_peers, df_data=fixed(df_data),
          msa=input_msa, year=fixed(2015), n_peers=input_n_peers,
                     fms=input_fms, outcomes=input_outcomes)

interactive(children=(Dropdown(description='MSA', index=237, options={'Abilene, TX ': 10180, 'Aguadilla-Isabel…

<function __main__.show_fms_peers(df_data, msa, year, n_peers, fms, outcomes)>

# Distinguishing features peers 

In [10]:
def show_disting_peers(df_data, msa, year, n_peers, n_feat):
    peers, fms = find.get_distinguishing_features_peers(df_data, msa, year, n_peers, n_feat)
    print(f"Comparison of {msa} and its peers for the 5 most distinguishing FMs")
    for i in range(0,5): 
        vis.quadrant_viz(df_data, msa, [msa]+peers, fms[i], save_fig=False, show=True)

interact_manual(show_disting_peers, df_data=fixed(df_data),
          msa=input_msa, year=input_year, n_peers=input_n_peers,
                     n_feat=input_n_features)



interactive(children=(Dropdown(description='MSA', index=237, options={'Abilene, TX ': 10180, 'Aguadilla-Isabel…

<function __main__.show_disting_peers(df_data, msa, year, n_peers, n_feat)>

# Top FMs peers

In [11]:
def show_top_fms_peers(df_data, msa, year, n_peers, n_fms):
    peers, fms = find.get_top_n_fms_peers(df_data, msa, year, n_peers, n_fms)
    print(f"Comparison of {msa} and its peers for the 5 most present FMs")
    for i in range(0,5): 
        vis.quadrant_viz(df_data, msa, [msa]+peers, fms[i], save_fig=False, show=True)

interact_manual(show_top_fms_peers, df_data=fixed(df_data),
          msa=input_msa, year=input_year, n_peers=input_n_peers,
                     n_fms=input_n_features)



interactive(children=(Dropdown(description='MSA', index=237, options={'Abilene, TX ': 10180, 'Aguadilla-Isabel…

<function __main__.show_top_fms_peers(df_data, msa, year, n_peers, n_fms)>